In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Importa arquivos
# Descomente pra usar no Google Colab
# from google.colab import files
# import os.path

arq_leis = 'leis.json'

# if (not os.path.isfile(arq_leis)):
#     uploaded = files.upload()

Saving leis.json to leis.json


In [3]:
leis = pd.read_json(arq_leis)
leis.drop(['documento'], inplace=True, axis=1)
print(leis.info())
print(leis.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6033 entries, 0 to 6032
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   titulo     6033 non-null   object
 1   categoria  6033 non-null   object
 2   resumo     6033 non-null   object
 3   texto      6033 non-null   object
dtypes: object(4)
memory usage: 188.7+ KB
None
titulo       6033
categoria       8
resumo       4961
texto        6029
dtype: int64


In [4]:
leis

,titulo,categoria,resumo,texto
0,"DECRETO Nº 8854, de 28 de fevereiro de 2013.",Decretos,DELEGA COMPETÊNCIA À SECRETARIA MUNICIPAL DE P...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
1,"DECRETO Nº 8849, de 25 de fevereiro de 2013.",Decretos,ABRE CRÉDITO SUPLEMENTAR AO ORÇAMENTO DO MUNIC...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
2,"DECRETO Nº 8853, de 27 de fevereiro de 2013.",Decretos,NOMEIA MEMBROS DO CONSELHO MUNICIPAL DE DESENV...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
3,"DECRETO Nº 8967, de 17 de julho de 2013",Decretos,ALTERA O QUADRO DE DETALHAMENTO DE DESPESA DO ...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
4,"DECRETO Nº 8982, de 30 de julho de 2013",Decretos,AUTORIZA O FUNCIONAMENTO DE ESTABELECIMENTOS C...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
...,...,...,...,...
6028,RESOLUÇÃO Nº 125/1980,Resoluções,DISPÕE SOBRE A CONCESSÃO DE TÍTULO DE CIDADÃO ...,Faço saber que a Câmara Municipal aprovou e eu...
6029,RESOLUÇÃO Nº 403/2003,Resoluções,AUTORIZA A MESA DIRETIVA DO PODER LEGISLATIVO ...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."
6030,RESOLUÇÃO Nº 492/2014,Resoluções,INSTITUI A SEGUNDA SEMANA DO MÊS DE AGOSTO EM ...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."
6031,RESOLUÇÃO Nº 382/2001,Resoluções,CRIA A MEDALHA VEREADOR DIVAL FIGUEIREDO MACHA...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."


In [5]:
# Exemplo de texto de lei
leis.loc[len(leis)-1, 'texto']

'A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, na conformidade do artigo 70, Inciso V, da Lei Municipal nº37, de 05 de Abril de 1990 e, artigos 287, § 2º e, 420, do Regimento Interno, promulga a seguinte Resolução:\n\nArt. 1ºDê-se aos dispositivos abaixo mencionados, da Resolução nº393/2002 - Regimento Interno, as seguintes redações:\n\n"Art. 7º A Mesa Diretora da Câmara compor-se-á do Presidente, Primeiro e Segundo Secretários, com mandato de 02 ( dois ) anos, admitida a recondução para a eleição subsequente.\n\n§ 4º Se, hora regimental, não estiver presente o Presidente, abrirá os trabalhos o Vice-Presidente ou, na falta deste, o Primeiro ou Segundo Secretários, na sequência, ou ainda, caso estes não estejam presentes, o Vereador mais votado nas eleições municipais."\n\n"Art. 33 Compete, privativamente, ao Vice-Presidente:"\n\n"Art. 36 ...\n\nI - ...\n\ne) acompanhar e supervisionar a Ata da Sessão, proceder a sua leitura e assiná-la depois do Presidente e do Vice-Presiden

In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import unicodedata

def limpa_texto(text, string=True) -> str or list:
    if (type(text) == float):
        return ''

    # Remove pontuacao, digitos e espacos
    text = ' '.join(re.findall(r'\b[a-zÀ-ú]+\b', text.lower()))
    
    # Remove acentos, cedilhas etc
    nfkd_form = unicodedata.normalize('NFKD', text)
    text = ''.join([char for char in nfkd_form if not unicodedata.combining(char)])

    # Remove stopwords
    my_stopwords = stopwords.words('portuguese')
    my_words = ['feira', 'santana', 'art', 'municipal', 'lei', 'r', 
    'prefeito', 'câmara', 'municipio', 'data', 'seguinte', 'disposições',
    'estado', 'bahia', 'vigor', 'secretário', 'decreto', 'projeto', 
    'iii', 'i', 'ii',  'contrário', 'presidente', 'artigo',
    'faço', 'parágrafo', 'executivo', 'gabinete', 'único', 'sanciono', 
    'desta', 'v', 'iv', 'autoria', 'através', 'deste', 'vice', 'autor',
    'qualquer', 'b', 'decretou', 'execução', 'sobre', 'das', 'decorrentes',
    'decreta', 'resolução', 'geral', 'uso', 'ato', 'diretiva', 'exercício',
    'seguintes', 'meio', 'm', 'c', 'correrão', 'g', 'n', 'w', 'k', 'l', 's',
    't', 'z', 'd', 'j', 'p', 'y', 'f', 'u', 'x', 'q', 'h']
    my_stopwords = my_stopwords + my_words

    text = [word for word in text.split() if word not in my_stopwords]

    if string:
        return ' '.join(text)
    else:
        return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Semelhança de documentos

In [11]:
leis['texto_limpo'] = leis['texto'].apply(limpa_texto)

In [12]:
# Constrói matriz de documentos
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(leis['texto_limpo'])
X

<6033x28875 sparse matrix of type '<class 'numpy.int64'>'
	with 534596 stored elements in Compressed Sparse Row format>

In [13]:
# Quero pegar o primeiro documento e achar qual documento mais similar a ele
# A partir daí, ler as leis pra ver se faz sentido, se são similares etc

# Pra achar os similares, computar a similaridade do cosseno
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# dense output pra conferir o resultado
cos_sim = cosine_similarity(X, dense_output=True)
cos_sim_argsort = np.argsort(cos_sim)

In [14]:
# cos_dist guarda uma matriz de similaridades cosseno
# Na primeira linha da matriz, estão as similaridades do primeiro texto pra todos os outros
# Capturar os 3 textos mais similares (tirando ele mesmo) e ler
max_sim_index = cos_sim_argsort[:,-2]
max_sim = [cos_sim[i, ind] for i, ind in enumerate(max_sim_index)] 

max_sim_overall = np.max(max_sim)
print(f'Similaridade maxima dos docs: {max_sim_overall}')

orig_text_ind = np.argmax(max_sim)
text_mais_sem_ind = max_sim_index[orig_text_ind]

Similaridade maxima dos docs: 1.0000000000000009


In [15]:
# Estatistica das maiores semelhanças
print(np.median(max_sim))
print(np.mean(max_sim))
print(np.std(max_sim))

0.7666518779999274
0.7452388636074964
0.20597156342823503


In [16]:
def print_lei_mais_sem(orig_idx : int, sem_idx: int):
    print(f'- - - LEI COMPARADA {orig_idx}: - - -\n\n')
    print(leis.loc[orig_idx, 'texto'])
    print(f'\n\n- - - LEI MAIS SEMELHANTE {sem_idx} - - -\n\n')
    print(leis.loc[sem_idx, 'texto'])

print_lei_mais_sem(orig_text_ind, text_mais_sem_ind)

- - - LEI COMPARADA 13: - - -


O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, no uso de suas atribuições legais e com base na autorização contida na Lei Nº3.350, de 11 de dezembro de 2012, artigo 6º, § 1º, DECRETA:

Art. 1ºFica aberto Crédito Suplementar ao Orçamento do Município no valor de R$ 10.950.000,00 (dez milhões, novecentos e cinquenta mil reais), conforme detalhamento abaixo:_____________________________________________________
| CLASS.|  PROGRAMÁTICA |ECONÔMICA|FONTE| VALOR (R$)  |
| INST. |               |         |     |             |
|=======|===============|=========|=====|=============|
|11.1111|10.122.025.2051|3.3.90.39|  002|10.950.000,00|
|-------|---------------|---------|-----|-------------|
|       |               |         |TOTAL|10.950.000,00|
|_______|_______________|_________|_____|_____________| * tabela formatada pela equipe técnica do LeisMunicipais.com.br
Art. 2ºOs recursos disponíveis para acorrer às despesas decorrentes do presente crédito s

Opa! Lei 13 e Lei 118 são a mesma lei, com 3 dias de diferença. Por que existe isso?

In [17]:
# Calcular similaridade usando TF-IDF agora
# Comparar com a mesma lei original anterior
transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(X)

# dense output pra conferir o resultado
cos_sim_tfidf = cosine_similarity(X_tfidf, dense_output=True)
cos_sim_tfidf_idx = np.argsort(cos_sim_tfidf)

X_tfidf

<6033x28875 sparse matrix of type '<class 'numpy.float64'>'
	with 534596 stored elements in Compressed Sparse Row format>

In [18]:
text_sem_ind = cos_sim_tfidf_idx[orig_text_ind, -2]
sem_tfidf = cos_sim_tfidf[orig_text_ind, text_sem_ind]

print(f'Para o texto original, a semelhanca do mais semelhante e: {sem_tfidf}')
print_lei_mais_sem(orig_text_ind, text_sem_ind)

Para o texto original, a semelhanca do mais semelhante e: 1.0
- - - LEI COMPARADA 13: - - -


O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, no uso de suas atribuições legais e com base na autorização contida na Lei Nº3.350, de 11 de dezembro de 2012, artigo 6º, § 1º, DECRETA:

Art. 1ºFica aberto Crédito Suplementar ao Orçamento do Município no valor de R$ 10.950.000,00 (dez milhões, novecentos e cinquenta mil reais), conforme detalhamento abaixo:_____________________________________________________
| CLASS.|  PROGRAMÁTICA |ECONÔMICA|FONTE| VALOR (R$)  |
| INST. |               |         |     |             |
|=======|===============|=========|=====|=============|
|11.1111|10.122.025.2051|3.3.90.39|  002|10.950.000,00|
|-------|---------------|---------|-----|-------------|
|       |               |         |TOTAL|10.950.000,00|
|_______|_______________|_________|_____|_____________| * tabela formatada pela equipe técnica do LeisMunicipais.com.br
Art. 2ºOs recursos disponív

In [19]:
# Ok, mostraram a mesma lei!
# A semelhanca usando tfidf eh menor
# Será que sempre mostram a mesma lei ou existe alguma dif?
# Se existe diff, qual o melhor metodo? Contando ou tfidf?
max_sim_idx_tfidf = cos_sim_tfidf_idx[:,-2]

idx_iguais = (max_sim_index == max_sim_idx_tfidf)
print(idx_iguais)

[ True False  True ... False  True  True]


In [20]:
# Os semelhantes pra segunda lei e pra anti-penultima sao diferentes
# Vamos dar uma olhada
print('Segunda lei e lei mais semelhante de acordo com count:\n\n')
print_lei_mais_sem(1, max_sim_index[1])

Segunda lei e lei mais semelhante de acordo com count:


- - - LEI COMPARADA 1: - - -


O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, no uso de suas atribuições legais e com base na autorização contida na Lei Nº3.350, de 11 de dezembro de 2012, artigo 6º, § 1º, DECRETA:

Art. 1ºFica aberto Crédito Suplementar ao Orçamento do Município no valor de R$ 199.000,00 (cento e noventa e nove mil reais), conforme detalhamento abaixo:__________________________________________________
| CLASS.|  PROGRAMÁTICA |ECONÔMICA|FONTE|VALOR (R$)|
| INST. |               |         |     |          |
|=======|===============|=========|=====|==========|
|13.1313|15.451.036.2097|4.4.90.92| 0000|190.000,00|
|-------|---------------|---------|-----|----------|
|21.2127|14.422.002.2223|3.3.90.39| 0000|  9.000,00|
|-------|---------------|---------|-----|----------|
|       |               |         |TOTAL|199.000,00|
|_______|_______________|_________|_____|__________| * tabela formatada pela equipe 

In [21]:
print('Segunda lei e lei mais semelhante de acordo com tfidf:\n\n')
print_lei_mais_sem(1, max_sim_idx_tfidf[1])

Segunda lei e lei mais semelhante de acordo com tfidf:


- - - LEI COMPARADA 1: - - -


O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, no uso de suas atribuições legais e com base na autorização contida na Lei Nº3.350, de 11 de dezembro de 2012, artigo 6º, § 1º, DECRETA:

Art. 1ºFica aberto Crédito Suplementar ao Orçamento do Município no valor de R$ 199.000,00 (cento e noventa e nove mil reais), conforme detalhamento abaixo:__________________________________________________
| CLASS.|  PROGRAMÁTICA |ECONÔMICA|FONTE|VALOR (R$)|
| INST. |               |         |     |          |
|=======|===============|=========|=====|==========|
|13.1313|15.451.036.2097|4.4.90.92| 0000|190.000,00|
|-------|---------------|---------|-----|----------|
|21.2127|14.422.002.2223|3.3.90.39| 0000|  9.000,00|
|-------|---------------|---------|-----|----------|
|       |               |         |TOTAL|199.000,00|
|_______|_______________|_________|_____|__________| * tabela formatada pela equipe 

- - - -

Ok, sao bastante similares ainda assim.

Acho que Count se saiu um pouco melhor porque o valor da segunda lei eh mais parecido com a primeira e ambas sao publicacao fevereiro, enquanto a tfidf eh publicacao abril.

Vamos olhar a antepenultima lei:

In [22]:
print('Antpenultima lei e lei mais semelhante de acordo com count:\n\n')
orig_idx = len(leis) - 3
print_lei_mais_sem(orig_idx, max_sim_index[-3])

Antpenultima lei e lei mais semelhante de acordo com count:


- - - LEI COMPARADA 6030: - - -


A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, na conformidade do artigo 70, inciso V, da Lei Municipal nº37, de 05 de Abril de 1990, e artigos 274, § 2º e, 400, do Regimento Interno, e do Projeto de Resolução nº 674/2014, de autoria da Edil Gerusa Maria Bastos S. Sampaio, promulga a seguinte Resolução:

Art. 1ºFica instituída, no âmbito da Câmara Municipal de Feira de Santana, Estado da Bahia, a "Semana em comemoração ao DIA dos Pais, que será na segunda semana do mês de Agosto de cada ano.

Parágrafo Único - Para efeito do exposto no caput deste artigo, esta Casa realizará atividades comemorativas alusivas ao período dedicado à Semana em comemoração ao "Dia dos Pais", externando o direito a cada Parlamentar homenagear um Pai, sendo ele figura paternal, ou genitor de uma pessoa.

Art. 2ºAs despesas decorrentes desta resolução correrão por conta de verba existente na Secretaria da 

In [23]:
print('Antpe lei e lei mais semelhante de acordo com TFIDF:\n\n')
print_lei_mais_sem(orig_idx, max_sim_idx_tfidf[-3])

Antpe lei e lei mais semelhante de acordo com TFIDF:


- - - LEI COMPARADA 6030: - - -


A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, na conformidade do artigo 70, inciso V, da Lei Municipal nº37, de 05 de Abril de 1990, e artigos 274, § 2º e, 400, do Regimento Interno, e do Projeto de Resolução nº 674/2014, de autoria da Edil Gerusa Maria Bastos S. Sampaio, promulga a seguinte Resolução:

Art. 1ºFica instituída, no âmbito da Câmara Municipal de Feira de Santana, Estado da Bahia, a "Semana em comemoração ao DIA dos Pais, que será na segunda semana do mês de Agosto de cada ano.

Parágrafo Único - Para efeito do exposto no caput deste artigo, esta Casa realizará atividades comemorativas alusivas ao período dedicado à Semana em comemoração ao "Dia dos Pais", externando o direito a cada Parlamentar homenagear um Pai, sendo ele figura paternal, ou genitor de uma pessoa.

Art. 2ºAs despesas decorrentes desta resolução correrão por conta de verba existente na Secretaria da Casa.



Ok, nessa o TF-IDF se sai bem melhor.

A original trata de semana de dia dos pais.

A que o count pegou trata de dia do agente comunitario e cita coisas como saude, endemia, despesas, que não ficaram muit claras pra mim o que são.

Já o TF-IDF trata de semana da familia. Não só familia é parecido com pai, mas tb é uma semnaa. E o texto da lei tb é sobre uma semana comemorativa. 

Vamos ver se existem mais diferenças e quais são estas diferenças:

In [24]:
difs = [i for i, igual in enumerate(idx_iguais) if not igual]
print(len(difs))

3161


Então pra mais da metade das leis Count e TF-IDF retornam resultados diferentes. Será que de TF-IDF pra vetor de palavras vai ter essa diferença toda também? Uma questão. 

Voltando ao que interessa, vamo dar uma olhada em mais algumas leis pra continuar a comparação entre Count e TF-IDF.

Vamo sortear indices aleatorios desse vetor e ler as leis que eles representam e as similaridades

In [25]:
num_leis_sorteadas = 10
idx_dif = np.random.randint(0, high=len(difs)-1, size=num_leis_sorteadas)
sorteadas = [difs[i] for i in idx_dif]
print(sorteadas)

[2093, 1550, 337, 4629, 5013, 2373, 1516, 2534, 2458, 250]


In [26]:
# Exibir as 10 leis:
for i in sorteadas:
    print(f'\n\nLei numero {i}\nPAR UTILIZANDO COUNT:\n\n')
    print_lei_mais_sem(i, max_sim_index[i])
    print('\n\nPAR UTILIZANDO TF-IDF:\n\n')
    print_lei_mais_sem(i, max_sim_idx_tfidf[i])



Lei numero 2093
PAR UTILIZANDO COUNT:


- - - LEI COMPARADA 2093: - - -


O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, ESTADO DA BAHIA, FAÇO saber que a Câmara Municipal decreta e eu sanciono a seguinte Lei:

Art. 1ºO artigo 64 e o parágrafo 1º da Lei870, de 29 de dezembro de 1979, passam a vigorar com a seguinte redação:

"Art. 64 - Os pedidos de isenção deverão ser renovados em cada exercício até o dia 30 de agosto, salvo as isenções concedidas por prazo determinado.

§ 1º VETADO."

Art. 2ºEsta Lei entrará em vigor na data de sua publicação, revogadas as diposições em contrário.

Prefeitura Municipal de Feira de Santana, em 04 de julho de 1991.

DR. COLBERT MARTINS DA SILVA
Prefeito

Autor: Rubens Carvalho


- - - LEI MAIS SEMELHANTE 1540 - - -


O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, ESTADO DA BAHIA, FAÇO saber que a Câmara Municipal decreta e eu sanciono a seguinte Lei:

Art. 1ºFicam revogados os incisos I e II, o parágrafo 1º, alíneas "A", "B" e "C" do artigo 19, e inciso I do a

### Lei 1018
Lei 1018 é sobre proibição de homenagens a condenados por corrupção. Count trouxe uma lei sobre tornar uma associação pública. TF-IDF trouxe uma lei sobre evento de comemoração de adoção animal. Todas duas erraram. TF-IDF chegou mais perto? Difícil dizer

### Lei 5776
Lei 5776 sobre pagamento servidor público. Count trouxe: leitura da bíblia na abertura da câmara. TF-IDF: aposentadoria diretor valor vencimento etc. Ambas as leis parecem ter sido trazidas como semelhantes pq dos nomes próprios contidos nas leis.

### Lei 2789
Lei 2789 (mil anos da revolução francesa) sobre obrigatoriedade de um servidor formado em primeiros socorros em escolas. Count: faço saber inkaba instituto de karate. TF-IDF: faço saber associação estrela jaco. Novamente as semelhanças são os nomes próprios nas leis.

### Lei 1772
Lei 1772: faço saber sindicato trabalhadores rurais. Count: faço saber associação profissionais sexo. TF-IDF: faço saber associação pequenos agricultures apaeb. A rua da sede é a mesma da lei comparada. 

TF-IDF se saiu melhor nessa. Os nomes das pessoas em Count eram os mesmos da Lei, mas em TF-IDF não. O fator decisivo aqui foi o nome da rua, que era o mesmo. Ponto pra TF-IDF.

### Lei 503
Lei 530: faço saber igreja ministerio pentecostal fogo gloria. rua volta redonda bairro campo limpo.
Count: faço saber instituto nobre sede rodovia br km cis. nomes das pessoas iguais.
TF-IDF: faço saber igreja evangelica pentecostal monte carmelo rua espassonavel bairro george americo. prefeitos diferentes.

### Lei 4810
Lei: comenda. nomes: godofredo rebell figueiredo filho, raymundo luiz oliveira lopes. 
Count: comenda. nomes: godofredo rebello figueiredo filho, nilton bellas vieira.
TF-IDF: comenda. nomes: godofredo rebello figueiredo filho, raimundo antonio carneiro pinto.

As duas acertaram. TF-iDF nome mais parecido? Não deve influenciar...

### Lei 5238 
Lei 5238: promulgação de novas vias públicas. A via por TF-IDF passa por mais ruas semelhantes.

### Lei 5383
Lei promulga academia de ginástica. Count: promulga empresas serviço funerario. TF-IDF: promulga novos aparelhos de ginástica.

### Outras
As outras leis eram: _visualizar legislativo ba_. Ambas trouxeram textos idênticos.

Ok! Massa! Funciona!

Pelos resultados acima, TF-IDF se saiu melhor. Inclusive pra retornar semelhança por nomes de ruas e de bairros, que é o que a gente quer pras buscas.

Mesmo com 28k features, o resultado foi bastante rápido. Caso tivessemos um corpus maior, poderíamos ainda usar PCA pra reduzir as dimensões e ainda assim calcular a similaridade mantendo as relações entre os documentos

Surgiram também duas questões: será que dá pra clusterizar as leis? Por ex, leis de faço saber, de comendas, etc... Será que formam clusters coesos? Vale visualizar. - caderno cluster_leis

A segunda é: então só precisa de TF-IDF pra fazer uma busca por ruas e bairros? Não precisa de NER? Vamos investigar pra ver isso também. - caderno buscador_leis

## Outras opções
### Indexar
Há outras formas de indexar os documentos e de recuperar, também simples. Uma outra forma de indexar, por exemplo, é fazer um vetor pra cada palavra contando as palavras vizinhas. E depois, o vetor do documento seria a soma dos vetores das palavras. É uma forma interessante porque pode gerar visualizações interessantes entre a similaridade das palavras. Por exemplo, no corpus das Leis Municipais, a quais palavras EDUCAÇÃO mais se assemelha? Ou SAÚDE? Etc.

Outra forma é contar n-gramas - por exemplo, bi-gramas: duas palavras juntas formando um token. Dessa forma, você possui uma matriz maior e de certa forma uma relação entre a sequencialidade das palavras, que pode ser útil pra nomes de pessoas e bairros, como citado acima.

### Recuperar
Outra forma de recuperar é por local sensitive hashing. Divide em vários planos múltiplas vezes e retorna os resultados que estão na mesma região da query. No entanto, o corpus não é grande o suficiente pra precisar essa estratégia, que é mais pra grandes corpora. O método acima (calcular a simlaridade cosseno e retornar os maiores valores) é rápido o suficiente pra parecer instantâneo. Talvez com uma demanda mais alta pelo servidor venha a necessidade de aumentar a velocidade da busca, porém por enquanto não é o caso. Mais sobre recuperação: Google lançou novo método e uma lib pra isso agora, dia 28 de Julho: https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html

### Avaliação
Com múltiplas formas de indexar e recuperar vem o dilema: como avaliar se uma é melhor que a outra? Repetir o processo acima pra todas as opções? Isto é, mostrar N melhores resultados e comparar manualmente? Ou colocar labels em algumas leis? Ex: essa lei trata disso, com tais entidades. Checar formas de avaliação. Se tivesse em produção, podia avaliar por CTR por ex, mas não é o caso

In [27]:
# Descrevendo a palavra pelos seus vizinhos
texto_limpo = ' '.join(leis['texto_limpo'].tolist())
texto_limpo = texto_limpo.split()
palavras_unicas = set(texto_limpo)
print(len(texto_limpo))
len(palavras_unicas)

972616


28875

In [28]:
# Transformar nosso set pra um dicionário de indices
idx_palavras = {}
for i, palavra in enumerate(palavras_unicas):
    idx_palavras[palavra] = i

In [29]:
# Criando matriz do embedding
vetores = np.zeros((len(palavras_unicas), len(palavras_unicas)), dtype=np.int16)
vetores

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

In [30]:
# Definir ate qual distancia utilizar
vizinhanca = 2
for idx, palavra in enumerate(texto_limpo):
    for i in range(1, vizinhanca):
        pal_vizinha = texto_limpo[idx+i]

        idx_pal = idx_palavras[palavra]
        idx_vizinha = idx_palavras[pal_vizinha]

        vetores[idx_pal, idx_vizinha] += 1
        vetores[idx_vizinha, idx_pal] += 1
    if (idx == len(texto_limpo) - vizinhanca):
        break
vetores

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

In [31]:
# Matriz tem que ser espassa, se não na hora de calcular
# similaridade gasta muita memoria

from scipy.sparse import csr_matrix
vetores = csr_matrix(vetores)
vetores

<28875x28875 sparse matrix of type '<class 'numpy.int16'>'
	with 658329 stored elements in Compressed Sparse Row format>

In [32]:
# Ok, temos nossas representacoes pras palavras
# Como um teste de sanidade, vamos ver 
# as palavras mais proximas de 10 palavras aleatorias
cos_sim_palavras = cosine_similarity(vetores, dense_output=False)
cos_sim_palavras


<28875x28875 sparse matrix of type '<class 'numpy.float64'>'
	with 76446099 stored elements in Compressed Sparse Row format>

In [33]:
# palavras_semelhantes = np.asanyarray(cos_sim_palavras)
# palavras_semelhantes = np.argsort(palavras_semelhantes, axis=1)
# palavras_semelhantes = np.sort(cos_sim_palavras)
palavras_semelhantes = np.argsort(cos_sim_palavras[0].toarray())
palavras_semelhantes

array([[14437, 18921, 18920, ..., 14528, 14974,     0]])

In [34]:
idx_palavras_rand = np.random.randint(len(palavras_unicas), size=10)

def mostra_palavras_semelhantes(idx: int, semelhantes: list, lista_palavras: list):
    pal = lista_palavras[idx]
    print(f'Lista de palavras semelhantes a {pal} - {idx}:')
    for i in semelhantes:
        pal = lista_palavras[i]
        print(f'{pal} - {i}')
    print('\n- - - - - \n\n')

lista_palavras = list(idx_palavras.keys())
for idx in idx_palavras_rand:
    palavras_semelhantes = np.argsort(cos_sim_palavras[idx].toarray())
    semelhantes = palavras_semelhantes[0][-10:-1]
    mostra_palavras_semelhantes(idx, semelhantes, lista_palavras)

Lista de palavras semelhantes a funerais - 3755:
rela - 25787
oe - 18886
volumosa - 28313
diligenciando - 3698
equipando - 22118
aerofotogrametria - 22416
sondagem - 13553
portico - 2178
adjudicacoes - 13209

- - - - - 


Lista de palavras semelhantes a eeg - 4993:
concorrentemente - 744
discordancia - 27422
consorciar - 9462
estimulaa - 10918
instrumen - 21973
usada - 13383
paramentos - 12168
estimulacao - 1754
televisores - 7262

- - - - - 


Lista de palavras semelhantes a estomologia - 7844:
laboratorial - 22997
terapeutico - 3229
turma - 12594
compondo - 3565
imagem - 11763
radioisotopos - 1179
neonatal - 26241
socioterritorial - 27798
estomologia - 7844

- - - - - 


Lista de palavras semelhantes a digitalizada - 16781:
base - 16268
regras - 4471
implantar - 4372
restituicao - 4335
aeronaves - 4581
visual - 5228
comprometer - 20083
aero - 12379
ifr - 4252

- - - - - 


Lista de palavras semelhantes a providencias - 8126:
estatuindo - 7324
repassa - 11663
procuracoes - 1967
recupe

Neste corpus, pra algumas palavras, a hipótese distribucional parece funcionar bem, pra outras nem tanto, pra outras não funciona.

Semelhantes a "outorgar" temos: "permutar", "editar", "contratar", "doar", "conceder", "dispensar", "celebrar", "subscrever", "proibir". Embora a semântica (significado) não seja necessariamente próxima, todas as palavras são verbos, então a sintaxe é próxima. Semelhantes a "ibitita": "axixa", "ibirarema", "peritoro", "piracaia", "igarata", "erechim", "itaperuna", "piata", "vandinha". Todos parecem nomes de locais.

Existem casos horríveis. Semelhantes à "coesao" temos: "sedeso", "his", "ctps", "zeis", "pnas", "cgfmhis", "snhis", "acemas". O que significam essas palavras? Talvez seja útil melhorar a qualidade do pré-processamento pra melhorar na indexação. Semelhantes à "separando" temos: "agrossilvopastoris", "cemiteriais", "solidos", "molhados", "domiciliares", "volumosos", "baldios", "antecedencia", "dimensao".

Há casos mistos. Semelhantes `a "trasporte" (note o erro) temos: "meia" (talvez meia passagem?), "transporte" (a palavra correta aparece em segundo), "roletas", "vala" (?), "trafegos", "convencional" (?), "edificar" (?), "passageiros", "fretado".

Talvez o corpus seja pequeno demais pra encontrar as relações entre as palavras só contando? Há de se testar se não é melhor então trabalhar com vetores de palavras, mesmo aprendidos em um corpus pequeno. Segundo o paper "Don't count, predict! a systematic comparison of context-counting vs. context-predicting semantic vectors (2014) - Baroni, Dinu, Kruszeweski", predizer é melhor que contar. Há de se testar se neste nosso contexto isso também se verifica.

Por hora, vamos testar se a busca melhora ou não utilizando as palavras. Então vamos construir a representação das leis.

# Construindo representação das leis com base na hipotese distribucional

In [35]:
# Cada lei vai ser a soma dos vetores de suas palavras
# Usando np.zeros gasta muita memoria, mas csr_matrix eh muito lento
leis_vetores_palavras = np.zeros((len(leis['texto_limpo']), vetores.shape[1]))
for idx, lei in enumerate(leis['texto_limpo']):
    for palavra in lei.split():
        idx_palavra = idx_palavras[palavra]
        leis_vetores_palavras[idx] += vetores[idx_palavra]
leis_vetores_palavras

array([[ 1.,  0.,  4., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 2., 12.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  6.,  0., ...,  0.,  0.,  0.],
       [ 0., 21.,  0., ...,  0.,  0.,  0.],
       [ 0., 33., 17., ...,  0.,  0.,  0.]])

In [36]:
vetores_similaridade = cosine_similarity(leis_vetores_palavras)

In [37]:
mais_semelhantes_vetores = [idx for idx in np.argsort(vetores_similaridade)[:,-2]]

In [42]:
# Diferenca de vetores pra TFIDF eh bem maior.
# Vamos samplear algumas dessas diferencas e 
# mostrar uma lei q tanto TFIDF como Count erraram
diferencas_TFIDF = (mais_semelhantes_vetores != max_sim_idx_tfidf)
sum(diferencas_TFIDF)

5163

In [43]:
sorteadas = np.random.randint(len(diferencas_TFIDF), size=10)
for i in sorteadas:
    if diferencas_TFIDF[i]:
        print(f'\n\nLei numero {i}\nPAR UTILIZANDO VETOR:\n\n')
        print_lei_mais_sem(i, mais_semelhantes_vetores[i])
        print('\n\nPAR UTILIZANDO TF-IDF:\n\n')
        print_lei_mais_sem(i, max_sim_idx_tfidf[i])



Lei numero 3366
PAR UTILIZANDO VETOR:


- - - LEI COMPARADA 3366: - - -


O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, FAÇO saber que a Câmara Municipal, através do Projeto de Lei nº 5/2014, de autoria da Edil Gerusa Maria Bastos Sampaio, decretou e eu sanciono a seguinte Lei:

Art. 1ºFica considerado de utilidade pública o PROJETO IRMÃO SOLIDÁRIO - PROISO, com sede localizada à Rua Estrela do Sul, nº 200, Conjunto Stella Maria, no Bairro Muchila I, neste Município de Feira de Santana.

Art. 2ºEsta Lei entrará em vigor na data de sua publicação, revogadas as disposições em contrário.

Gabinete do Prefeito, 19 de Março de 2014.

JOSÉ RONALDO DE CARVALHO
Prefeito

MÁRIO COSTA BORGES
Chefe de Gabinete do Prefeito

Autor: Gerusa Maria Bastos Sampaio


- - - LEI MAIS SEMELHANTE 2145 - - -


O PREFEITO MUNCIPAL DE FEIRA DE SANTANA, Estado da Bahia, FAÇO saber que a Câmara Municipal, através do Projeto de Lei nº 135/2013, de autoria da Edil Gerusa Maria Bastos Sampaio, decreto

Parece que TF-IDF performa um pouco melhor na comparação de leis, pq traz resultados mais relevantes quando comparados nome de bairros e ruas. No entanto, a forma vetorizada parece ser boa pra reconhecer formatos da Lei em geral, uma especie de POS, reconhecendo que existe uma entidade alí ou verbo etc. Ao menos foi minha impressão. Cabe mais investigação a respeito.

Vale salientar que a qualidade dos vetores parece não estar tãão boa, tambéém. Vide a semelhança de palavras. Como fazer pra consertar isso? Seria muito interessante corrigir isso pra ver as palavras mais semelhantes àà educaçãão, saúúde etc e tambéém pra visualizar com tsne os clusters gerados a partir daíí.